In [1]:
import pyarrow as pa
import numpy as np
import numba

/Users/sinayoks/apps/miniconda3/envs/numbarrow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Create a string in memory and use numba to reverse the string.

In [2]:
toreverse = pa.array('abcd')
toreverse

[
  'a',
  'b',
  'c',
  'd'
]

Where is the array stored in memory? Depends on the array type. See ???

In [3]:
missing, offsets, data = toreverse.buffers()

In [4]:
np.asarray(missing).view(np.uint8)

array([15], dtype=uint8)

The offsets, for each character in the string, plus the first character of the next string. 

In [5]:
np.asarray(offsets).view(np.uint32).reshape(5, -1)

array([[0],
       [1],
       [2],
       [3],
       [4]], dtype=uint32)

In [6]:
np.asarray(data).view(np.uint8).reshape(4, -1)

array([[ 97],
       [ 98],
       [ 99],
       [100]], dtype=uint8)

In [16]:
@numba.jitclass([
    ('start', numba.uint32),
    ('end', numba.uint32),
    ('data', numba.uint8[:]),
])
class NumbaString:
    def __init__(self, data, start=0, end=None):
        if end is None:
            end = data.shape[0]

        self.data = data
        self.start = start
        self.end = end

    @property
    def size(self):
        return self.end - self.start

    def get_byte(self, i):
        return self.data[self.start + i]


def _make_string(obj):
    if isinstance(obj, str):
        data = obj.encode('utf8')
        data = np.asarray(memoryview(data))
        return NumbaString(data, 0, len(data))
    if instance(obj, py.StringArray):
        missing, offsets, data = toreverse.buffers()
        view = np.asarray(data).view(np.uint8)
        return NumbaString(data, 0, len(data))
    raise TypeError()


NumbaString.make = _make_string


In [17]:
toreverse

[
  'a',
  'b',
  'c',
  'd'
]

In [18]:
numba_string = NumbaString.make('abcd')

In [19]:
numba_string

In [63]:
def reverse_string(string):
    n = string.size    
    res = np.zeros(string.size, dtype=np.uint8)
    for i in range(n):
        res[i] = string.get_byte(n - i  - 1)
    return res
        

In [64]:
numba_string.data

array([ 97,  98,  99, 100], dtype=uint8)

In [65]:
reverse_string(numba_string)

array([100,  99,  98,  97], dtype=uint8)

In [66]:
numba_string.get_byte(7)

0